In [54]:
# Init
# choropleth
from __future__ import division
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import matplotlib.pyplot as plt
import seaborn as sns

# County level data retrieval

In [55]:
df = pd.read_csv('../data/2019_data/prepped_data_2019.csv', na_values=".")
print(df.columns.tolist())

['Unnamed: 0', 'Unnamed: 0_x', 'fips', 'total_population', 'log_total_population', 'population_density', 'total_workers', 'land_area', 'sqrt_land_area', 'aggregate_num_vehicles', 'aggregate_time_commute', 'aggregate_time_commute_drive_alone', 'aggregate_time_commute_carpool', 'aggregate_time_commute_public_transit', 'aggregate_time_commute_other_means', 'total_workers_drove_alone', 'total_workers_carpooled', 'total_workers_public_transit', 'total_workers_other_means', 'ttl_wrkr_cmute', 'wrkrs_drove_alone', 'wrkrs_drove_alone_below_100_poverty', 'pct_wrkrs_drove_alone_below_100_poverty', 'wrkrs_drove_alone_100_to_150_poverty', 'wrkrs_drove_alone_over_150_poverty', 'wrkrs_carpool', 'wrkrs_carpool_below_100_poverty', 'pct_wrkrs_carpool_below_100_poverty', 'wrkrs_carpool_100_to_150_poverty', 'wrkrs_carpool_over_150_poverty', 'wrkrs_public_transit', 'wrkrs_public_transit_below_100_poverty', 'pct_wrkrs_public_transit_below_100_poverty', 'wrkrs_public_transit_100_to_150_poverty', 'wrkrs_publi

# Data Cleanup involves combining data from two different years

In [56]:
# ##### Data Cleanup

# df = df.dropna(subset=['vehicles_per_worker',
#                                  'total_workers',
#                                  'land_area',
#                                  'pct_unemployed',
#                                  'average_commute_time_per_worker_drive_alone',
#                                  'pct_car_wrkr_cmute_01_to_14_min',
#                                  'pct_car_wrkr_cmute_15_to_29_min',
#                                  'pct_car_wrkr_cmute_30_to_44_min',
#                                  'pct_car_wrkr_cmute_44_to_59_min',
#                                  'pct_car_wrkr_cmute_over_60_min',
#                                  'total_population'])

# df['total_workers'] = df['total_workers']
# df['land_area'] = df['land_area']
# df['vehicles_per_worker'] = df['vehicles_per_worker']
# df['average_commute_time_per_worker_drive_alone'] = df['average_commute_time_per_worker_drive_alone']
# df['pct_car_wrkr_cmute_01_to_14_min'] = df['pct_car_wrkr_cmute_01_to_14_min']
# df['pct_car_wrkr_cmute_15_to_29_min'] = df['pct_car_wrkr_cmute_15_to_29_min']
# df['pct_car_wrkr_cmute_44_to_59_min'] = df['pct_car_wrkr_cmute_44_to_59_min']
# df['pct_car_wrkr_cmute_30_to_44_min'] = df['pct_car_wrkr_cmute_30_to_44_min']
# df['pct_car_wrkr_cmute_over_60_min'] = df['pct_car_wrkr_cmute_over_60_min']
# df['pct_unemployed'] = df['pct_unemployed']
# df['total_population'] = df['total_population']
# df['year'] = 2019

# OLS Regression - Multivariate

In [57]:
df = df
df.pct_walking_and_public_transit = df.pct_all_public_tspt_wrkr_cmute + df.pct_walked_wrkr_cmute
df.pct_walking_and_public_transit


C:\Users\cjong\AppData\Local\Temp\ipykernel_25460\3390946073.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.pct_walking_and_public_transit = df.pct_all_public_tspt_wrkr_cmute + df.pct_walked_wrkr_cmute


0       0.015675
1       0.006936
2       0.030587
3       0.001391
4       0.005852
          ...   
3215    0.019362
3216    0.044232
3217    0.025020
3218    0.020998
3219    0.038529
Length: 3220, dtype: float64

In [58]:
df = df
df['pct_walking_and_public_transit'] = df['pct_all_public_tspt_wrkr_cmute'] + df['pct_walked_wrkr_cmute']
df['pct_walking_and_public_transit']


0       0.015675
1       0.006936
2       0.030587
3       0.001391
4       0.005852
          ...   
3215    0.019362
3216    0.044232
3217    0.025020
3218    0.020998
3219    0.038529
Name: pct_walking_and_public_transit, Length: 3220, dtype: float64

### Thought process
If there are significances found between the percentage of people that commute to work by walking & public transit and the combination of cars per worker, 

Based on the research paper 
https://www.sciencedirect.com/science/article/pii/S2214367X22000333
"The results reveal that the choice of public transport as a daily commute mode is significantly affected by car ownership, distance to work, parking availability, and ticket prices"

In [59]:
# ols regression logic the following
# walking and public total pct is affected by the following
# average vehicles per worker
# 
lm_ols = smf.ols(formula = 'pct_walking_and_public_transit ~ \
                                total_population + \
                                aggregate_num_vehicles + \
                                pct_unemployed + \
                                land_area + \
                                cbp_total_employees + \
                                cbp_total_establishments',
                                data = df).fit()
print (lm_ols.summary())

                                  OLS Regression Results                                  
Dep. Variable:     pct_walking_and_public_transit   R-squared:                       0.388
Model:                                        OLS   Adj. R-squared:                  0.387
Method:                             Least Squares   F-statistic:                     331.1
Date:                            Sun, 15 Jan 2023   Prob (F-statistic):               0.00
Time:                                    15:45:35   Log-Likelihood:                 5384.7
No. Observations:                            3139   AIC:                        -1.076e+04
Df Residuals:                                3132   BIC:                        -1.071e+04
Df Model:                                       6                                         
Covariance Type:                        nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.97

In [60]:
# ols regression logic the following
# walking and public total pct is affected by the following
# average vehicles per worker
# 
lm_ols = smf.ols(formula = 'pct_walking_and_public_transit ~ \
                                total_population + \
                                aggregate_num_vehicles + \
                                roads_length + \
                                intersections + \
                                pct_unemployed + \
                                land_area + \
                                cbp_total_employees + \
                                cbp_total_establishments',
                                data = df).fit()
print (lm_ols.summary())

                                  OLS Regression Results                                  
Dep. Variable:     pct_walking_and_public_transit   R-squared:                       0.403
Model:                                        OLS   Adj. R-squared:                  0.401
Method:                             Least Squares   F-statistic:                     263.6
Date:                            Sun, 15 Jan 2023   Prob (F-statistic):               0.00
Time:                                    15:45:35   Log-Likelihood:                 5422.2
No. Observations:                            3139   AIC:                        -1.083e+04
Df Residuals:                                3130   BIC:                        -1.077e+04
Df Model:                                       8                                         
Covariance Type:                        nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.97

In [61]:
# ols regression logic the following
# walking and public total pct is affected by the following
# average vehicles per worker
# 
lm_ols = smf.ols(formula = 'pct_all_public_tspt_wrkr_cmute ~ \
                                vehicles_per_worker + \
                                roads_length + \
                                intersections + \
                                total_population + \
                                average_commute_time_per_worker_drive_alone + \
                                pct_unemployed + \
                                land_area + \
                                cbp_total_employees + \
                                cbp_total_establishments',
                                data = df).fit()
print (lm_ols.summary())

                                  OLS Regression Results                                  
Dep. Variable:     pct_all_public_tspt_wrkr_cmute   R-squared:                       0.615
Model:                                        OLS   Adj. R-squared:                  0.613
Method:                             Least Squares   F-statistic:                     245.2
Date:                            Sun, 15 Jan 2023   Prob (F-statistic):          1.02e-278
Time:                                    15:45:35   Log-Likelihood:                 2943.4
No. Observations:                            1389   AIC:                            -5867.
Df Residuals:                                1379   BIC:                            -5814.
Df Model:                                       9                                         
Covariance Type:                        nonrobust                                         
                                                  coef    std err          t      P>|t|   

In [62]:
# ols regression logic the following
# walking and public total pct is affected by the following
# average vehicles per worker
# 
lm_ols = smf.ols(formula = 'pct_walking_and_public_transit ~ \
                                vehicles_per_worker + \
                                roads_length + \
                                intersections + \
                                total_population + \
                                average_commute_time_per_worker_drive_alone + \
                                pct_unemployed + \
                                land_area + \
                                cbp_total_employees + \
                                cbp_total_establishments',
                                data = df).fit()
print (lm_ols.summary())

                                  OLS Regression Results                                  
Dep. Variable:     pct_walking_and_public_transit   R-squared:                       0.770
Model:                                        OLS   Adj. R-squared:                  0.769
Method:                             Least Squares   F-statistic:                     513.2
Date:                            Sun, 15 Jan 2023   Prob (F-statistic):               0.00
Time:                                    15:45:35   Log-Likelihood:                 2886.2
No. Observations:                            1389   AIC:                            -5752.
Df Residuals:                                1379   BIC:                            -5700.
Df Model:                                       9                                         
Covariance Type:                        nonrobust                                         
                                                  coef    std err          t      P>|t|   

In [63]:
# ols regression logic the following
# walking and public total pct is affected by the following
# average vehicles per worker
# 
lm_ols = smf.ols(formula = 'pct_walking_and_public_transit ~ \
                                vehicles_per_worker + \
                                roads_length + \
                                intersections + \
                                total_population + \
                                average_commute_time_per_worker_drive_alone + \
                                pct_unemployed + \
                                land_area + \
                                cbp_total_employees + \
                                cbp_total_establishments',
                                data = df).fit()
print (lm_ols.summary())

                                  OLS Regression Results                                  
Dep. Variable:     pct_walking_and_public_transit   R-squared:                       0.765
Model:                                        OLS   Adj. R-squared:                  0.763
Method:                             Least Squares   F-statistic:                     640.5
Date:                            Sun, 15 Jan 2023   Prob (F-statistic):               0.00
Time:                                    15:45:36   Log-Likelihood:                 2869.6
No. Observations:                            1389   AIC:                            -5723.
Df Residuals:                                1381   BIC:                            -5681.
Df Model:                                       7                                         
Covariance Type:                        nonrobust                                         
                                                  coef    std err          t      P>|t|   

In [64]:
# ols regression logic the following
# walking and public total pct is affected by the following
# average vehicles per worker

lm_ols = smf.ols(formula = 'pct_walking_and_public_transit ~ \
                                vehicles_per_worker + \
                                roads_length + \
                                intersections + \
                                total_population + \
                                average_commute_time_per_worker_drive_alone + \
                                pct_unemployed + \
                                land_area + \
                                cbp_total_employees + \
                                cbp_total_establishments',
                                data = df).fit()
print (lm_ols.summary())

                                  OLS Regression Results                                  
Dep. Variable:     pct_walking_and_public_transit   R-squared:                       0.770
Model:                                        OLS   Adj. R-squared:                  0.769
Method:                             Least Squares   F-statistic:                     513.2
Date:                            Sun, 15 Jan 2023   Prob (F-statistic):               0.00
Time:                                    15:45:36   Log-Likelihood:                 2886.2
No. Observations:                            1389   AIC:                            -5752.
Df Residuals:                                1379   BIC:                            -5700.
Df Model:                                       9                                         
Covariance Type:                        nonrobust                                         
                                                  coef    std err          t      P>|t|   

In [65]:
lm_ols = smf.ols(formula = 'pct_walking_and_public_transit ~ \
                                vehicles_per_worker + \
                                roads_length + \
                                intersections + \
                                total_population + \
                                pct_unemployed + \
                                land_area + \
                                cbp_total_employees + \
                                cbp_total_establishments \
                                ',
                                # average_commute_time_per_worker_drive_alone + \
data = df).fit()
print (lm_ols.summary())

                                  OLS Regression Results                                  
Dep. Variable:     pct_walking_and_public_transit   R-squared:                       0.698
Model:                                        OLS   Adj. R-squared:                  0.697
Method:                             Least Squares   F-statistic:                     904.9
Date:                            Sun, 15 Jan 2023   Prob (F-statistic):               0.00
Time:                                    15:45:36   Log-Likelihood:                 6493.6
No. Observations:                            3139   AIC:                        -1.297e+04
Df Residuals:                                3130   BIC:                        -1.291e+04
Df Model:                                       8                                         
Covariance Type:                        nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.97

In [66]:
lm_ols = smf.ols(formula = 'pct_walking_and_public_transit ~ \
                                vehicles_per_worker + \
                                roads_length + \
                                intersections + \
                                total_population + \
                                pct_unemployed + \
                                land_area ',
                                # average_commute_time_per_worker_drive_alone + \
data = df).fit()
print (lm_ols.summary())

                                  OLS Regression Results                                  
Dep. Variable:     pct_walking_and_public_transit   R-squared:                       0.696
Model:                                        OLS   Adj. R-squared:                  0.696
Method:                             Least Squares   F-statistic:                     1198.
Date:                            Sun, 15 Jan 2023   Prob (F-statistic):               0.00
Time:                                    15:45:36   Log-Likelihood:                 6463.1
No. Observations:                            3141   AIC:                        -1.291e+04
Df Residuals:                                3134   BIC:                        -1.287e+04
Df Model:                                       6                                         
Covariance Type:                        nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
--

# Comparing data for business establishment per county
## this is on top of total road density, road intersection density, total population, total vehicles
## include total time per commute method driving / carpooling vs walking & everything else

In [67]:
# lm_ols = smf.ols(formula = 'pct_walking_and_public_transit ~ \
#                                 vehicles_per_worker + \
#                                 roads_length + \
#                                 intersections + \
#                                 total_population + \
#                                 pct_unemployed + \
#                                 land_area ',
# data = df).fit()
# print (lm_ols.summary())

In [68]:
lm_ols = smf.ols(formula = 'pct_walking_and_public_transit ~ \
                                vehicles_per_worker + \
                                roads_length + \
                                intersections + \
                                total_population + \
                                pct_unemployed + \
                                land_area + \
                                ttl_households + \
                                land_area',
data = df).fit()
print (lm_ols.summary())

                                  OLS Regression Results                                  
Dep. Variable:     pct_walking_and_public_transit   R-squared:                       0.706
Model:                                        OLS   Adj. R-squared:                  0.705
Method:                             Least Squares   F-statistic:                     1074.
Date:                            Sun, 15 Jan 2023   Prob (F-statistic):               0.00
Time:                                    15:45:36   Log-Likelihood:                 6512.4
No. Observations:                            3141   AIC:                        -1.301e+04
Df Residuals:                                3133   BIC:                        -1.296e+04
Df Model:                                       7                                         
Covariance Type:                        nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
--

In [69]:
lm_ols = smf.ols(formula = 'pct_walking_and_public_transit ~ \
                                vehicles_per_worker + \
                                roads_length + \
                                intersections + \
                                total_population + \
                                pct_unemployed + \
                                land_area + \
                                ttl_households + \
                                land_area + \
                                cbp_total_establishments',
data = df).fit()
print (lm_ols.summary())

                                  OLS Regression Results                                  
Dep. Variable:     pct_walking_and_public_transit   R-squared:                       0.703
Model:                                        OLS   Adj. R-squared:                  0.702
Method:                             Least Squares   F-statistic:                     925.7
Date:                            Sun, 15 Jan 2023   Prob (F-statistic):               0.00
Time:                                    15:45:36   Log-Likelihood:                 6518.6
No. Observations:                            3139   AIC:                        -1.302e+04
Df Residuals:                                3130   BIC:                        -1.296e+04
Df Model:                                       8                                         
Covariance Type:                        nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.97

In [70]:
lm_ols = smf.ols(formula = 'pct_walking_and_public_transit ~ \
                                vehicles_per_worker + \
                                roads_length + \
                                intersections + \
                                total_population + \
                                pct_unemployed + \
                                land_area + \
                                ttl_households + \
                                land_area + \
                                cbp_total_establishments + cbp_total_employees',
data = df).fit()
print (lm_ols.summary())

                                  OLS Regression Results                                  
Dep. Variable:     pct_walking_and_public_transit   R-squared:                       0.703
Model:                                        OLS   Adj. R-squared:                  0.702
Method:                             Least Squares   F-statistic:                     823.9
Date:                            Sun, 15 Jan 2023   Prob (F-statistic):               0.00
Time:                                    15:45:36   Log-Likelihood:                 6520.5
No. Observations:                            3139   AIC:                        -1.302e+04
Df Residuals:                                3129   BIC:                        -1.296e+04
Df Model:                                       9                                         
Covariance Type:                        nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.97

In [71]:
lm_ols = smf.ols(formula = 'pct_walking_and_public_transit ~ \
                                vehicles_per_worker + \
                                roads_length + \
                                intersections + \
                                total_population + \
                                pct_unemployed + \
                                land_area + \
                                ttl_households + \
                                land_area + \
                                cbp_num_employees_5 + \
                                cbp_num_employees_5_9 + \
                                cbp_num_employees_10_19 + \
                                cbp_num_employees_20_49 + \
                                cbp_num_employees_50_99 + \
                                cbp_num_employees_100_249 + \
                                cbp_num_employees_250_499 + \
                                cbp_num_employees_500_999 + \
                                cbp_num_employees_1000',
data = df).fit()
print (lm_ols.summary())
# lm_ols = smf.ols(formula = 'pct_walking_and_public_transit ~ \
#                                 vehicles_per_worker + \
#                                 roads_length + \
#                                 intersections + \
#                                 total_population + \
#                                 pct_unemployed + \
#                                 land_area + \
#                                 ttl_households + \
#                                 land_area + \
#                                 cbp_num_employees_5 ',
# data = df).fit()
# print (lm_ols.summary())

                                  OLS Regression Results                                  
Dep. Variable:     pct_walking_and_public_transit   R-squared:                       0.720
Model:                                        OLS   Adj. R-squared:                  0.719
Method:                             Least Squares   F-statistic:                     502.0
Date:                            Sun, 15 Jan 2023   Prob (F-statistic):               0.00
Time:                                    15:45:36   Log-Likelihood:                 6612.2
No. Observations:                            3139   AIC:                        -1.319e+04
Df Residuals:                                3122   BIC:                        -1.309e+04
Df Model:                                      16                                         
Covariance Type:                        nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.9

In [72]:
lm_ols = smf.ols(formula = 'pct_walking_and_public_transit ~ \
                                vehicles_per_worker + \
                                roads_length + \
                                intersections + \
                                total_population + \
                                pct_unemployed + \
                                ttl_households + \
                                sqrt_land_area + \
                                cbp_total_establishments + \
                                cbp_num_employees_5 + \
                                cbp_num_employees_5_9 + \
                                cbp_num_employees_10_19 + \
                                cbp_num_employees_20_49 + \
                                cbp_num_employees_50_99 + \
                                cbp_num_employees_100_249 + \
                                cbp_num_employees_250_499 + \
                                cbp_num_employees_500_999 + \
                                cbp_num_employees_1000',
data = df).fit()
print (lm_ols.summary())

                                  OLS Regression Results                                  
Dep. Variable:     pct_walking_and_public_transit   R-squared:                       0.716
Model:                                        OLS   Adj. R-squared:                  0.714
Method:                             Least Squares   F-statistic:                     462.9
Date:                            Sun, 15 Jan 2023   Prob (F-statistic):               0.00
Time:                                    15:45:37   Log-Likelihood:                 6589.5
No. Observations:                            3139   AIC:                        -1.314e+04
Df Residuals:                                3121   BIC:                        -1.303e+04
Df Model:                                      17                                         
Covariance Type:                        nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.9

# LASSO REGRESSION

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
# from sklearn.linear_model import ElasticNet
# model = ElasticNet(alpha=1.0, l1_ratio=0.5)
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()

# X = df['pct_walking_and_public_transit']
# y = df[['vehicles_per_worker','roads_length','intersections','total_population','average_commute_time_per_worker_drive_alone','pct_unemployed','land_area']]

df = df.dropna(subset=['pct_walking_and_public_transit','vehicles_per_worker','roads_length','intersections','total_population','average_commute_time_per_worker_drive_alone','pct_unemployed','land_area','cbp_total_employees', 'cbp_total_establishments'])

X = df[['vehicles_per_worker','roads_length','intersections','total_population','average_commute_time_per_worker_drive_alone','pct_unemployed','land_area','cbp_total_employees', 'cbp_total_establishments']]
y = df['pct_walking_and_public_transit']

In [74]:

# Here, I standardize by X data using StandardScalar
X_train, X_test, y_train, y_test = train_test_split(X, y)
# scaler.fit(X_test)
# X_test_scaled = scaler.transform(X_test)
# scaler.fit(X_train)
# X_train_scaled = scaler.transform(X_train)

scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [75]:
lasso = Lasso().fit(X_train, y_train) # Fit the training data to a lasso regression line with alpha = 0.01 and 100,000 iterations

print("LASSO REGRESSION (UNSCALED DATA)") # LASSO is L1 penalty, which adds “absolute value of magnitude” of coefficient as penalty term to the loss function. Good for feature selection when you have a lot of features.
print("Training set score: {:.3f}".format(lasso.score(X_train, y_train)))
print("Test set score: {:.3f}".format(lasso.score(X_test, y_test)))

# Kfold cross validation
print("Mean Cross-Validation, Kfold: {:.3f}".format(np.mean(cross_val_score(lasso, X_train, y_train))))

lasso_unscaled = np.mean(cross_val_score(lasso, X_train, y_train)) # Will use later to compare results

# define model
model = Lasso(alpha=1.0)

# evaluate an lasso regression model on the dataset
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
dataframe = read_csv(url, header=None)
data = dataframe.values
X, y = data[:, :-1], data[:, -1]
# define model
model = Lasso(alpha=1.0)
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

LASSO REGRESSION (UNSCALED DATA)
Training set score: 0.250
Test set score: 0.169
Mean Cross-Validation, Kfold: -0.016
Mean MAE: 3.711 (0.549)


In [76]:
# load and summarize the housing dataset
from pandas import read_csv
from matplotlib import pyplot
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
dataframe = read_csv(url, header=None)
# summarize shape
print(dataframe.shape)
# summarize first few lines
print(dataframe.head())

(506, 14)
        0     1     2   3      4      5     6       7   8      9     10  \
0  0.00632  18.0  2.31   0  0.538  6.575  65.2  4.0900   1  296.0  15.3   
1  0.02731   0.0  7.07   0  0.469  6.421  78.9  4.9671   2  242.0  17.8   
2  0.02729   0.0  7.07   0  0.469  7.185  61.1  4.9671   2  242.0  17.8   
3  0.03237   0.0  2.18   0  0.458  6.998  45.8  6.0622   3  222.0  18.7   
4  0.06905   0.0  2.18   0  0.458  7.147  54.2  6.0622   3  222.0  18.7   

       11    12    13  
0  396.90  4.98  24.0  
1  396.90  9.14  21.6  
2  392.83  4.03  34.7  
3  394.63  2.94  33.4  
4  396.90  5.33  36.2  
